In [ ]:
import pandas as pd
import numpy as np
import config
data_df = pd.read_csv(config.train_data_path)
data_df.head()
label_df = data_df["label"]
feature_df = data_df.drop("label", axis=1)
print(feature_df.shape)
# 归一化处理
feature_df = feature_df/255.0
# 进行数据变换，变换成1*28*28(C*H*W)的图像输入形式
feature_df = feature_df.apply(lambda x:x.values.reshape(1,28,28), axis=1)
feature_df


In [ ]:
import torch
from torchvision import transforms
import numpy as np
import pandas as pd
from torch.utils.data import Dataset

import config

class TrainDataset(Dataset):
    def __init__(self,train_data_path):
        data_df = pd.read_csv(train_data_path)
        label_df = data_df["label"]
        feature_df = data_df.drop("label", axis=1)
        # 归一化处理
        feature_df = feature_df/255.0
        # 进行数据变换，变换成1*28*28(C*H*W)的图像输入形式
        feature_df = feature_df.apply(lambda x:x.values.reshape(1,28,28), axis=1)
        self.label_df = label_df
        self.images = feature_df
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.label_df)
    def __getitem__(self,index):
        return self.label_df[index],self.transform(self.images[index])

train_dataset = TrainDataset(config.train_data_path)
##可视化展示一下图片
import matplotlib.pyplot as plt
figure = plt.figure(figsize=(8, 8))
cols, rows = 4, 1
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(train_dataset.__len__(), size=(1,)).item()
    label, img = train_dataset[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(label)
    plt.axis("off") # 不显示坐标轴
    # squeeze是降维，去除度数1的维度，如灰度图像中C(通道)=1，绘制图像时不需要通道C这个维度，直接传递二维矩阵即可，所以将其去除，但这里由于img是28*28的矩阵不带C这个维度，所以不需要squeeze(),所以不需要squeeze在这里不起作用
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()


In [ ]:
import torch
import matplotlib.pyplot as plt
import config
epochs = []
loss_list = []

def scatter(epochs_list,loss_list):
    plt.figure(figsize=(10, 10), dpi=100)
    plt.scatter(epochs_list, loss_list)
    plt.show()
for i in range(1,2001):
    checkpoint = torch.load(config.model_path + "checkpoint_model_{}.pth".format(i), map_location='cpu')
    epochs.append(i)
    loss_list.append(checkpoint["total_loss"])
scatter(epochs,loss_list)
    